In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.3f}'.format


In [ ]:
import initial_condition


In [ ]:
global data_dir, output_dir, ex_dir
data_dir = '../../data/processed/internal/tech/'
ex_dir = '../../data/processed/external/schmoch/'


In [ ]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

extract_population = initial_condition.EXTRACT_POPULATION
# top_p_or_num = initial_condition.TOP_P_OR_NUM
top_p_or_num = ('p', 3)
region_corporation = initial_condition.REGION_CORPORATION
region_corporation = 'right_person_name'
applicant_weight = initial_condition.APPLICANT_WEIGHT

classification = initial_condition.CLASSIFICATION
classification = 'ipc3'
class_weight = initial_condition.CLASS_WEIGHT

input_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'
output_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'


In [ ]:
output_condition

In [ ]:
df = pd.read_csv(data_dir + f'{input_condition}.csv', encoding='utf-8', engine='python', sep=',')
df

In [ ]:
# 各期間
classification_df = pd.merge(c_df.groupby([f'{ar}_{year_style}_period', classification])[['reg_num']].sum().reset_index(drop=False), 
                        c_df.groupby([f'{ar}_{year_style}_period', classification])[[region_corporation]].nunique().reset_index(drop=False), 
                        on=[f'{ar}_{year_style}_period', classification], 
                        how='inner')
classification_df = pd.merge(classification_df, 
                      c_df[[f'{ar}_{year_style}_period', classification, 'ubiquity', 'tci']\
                          +[f'ki_{i}' for i in range(1, 20+1)]]\
                          .drop_duplicates(keep='first'), 
                      on=[f'{ar}_{year_style}_period', classification], 
                      how='inner')

# classification_df['reg_num'] = classification_df['reg_num'].astype(np.int64)
classification_df = pd.merge(classification_df, 
                            schmoch_df.rename(columns={'Field_number':classification}), 
                            on=[classification], 
                            how='inner')\
                            .drop(columns=[classification])\
                            .rename(columns={'Field_en':classification})
# display(classification_df)
# schmoch_df['ipc3'] = schmoch_df['IPC_code'].str[:3]
# schmoch_df = schmoch_df.drop_duplicates()
# schmoch_df
# classification_df = pd.merge(classification_df,
#                                 schmoch_df,
#                                 # on=['ipc3'],
#                                 on=['schmoch35'],
#                                 how='left')\
#                             .rename(columns={'Field_en':'schmoch35'})\
#                             .drop(columns=['IPC_code', 'Field_number'])\
#                             .drop_duplicates()
# for period in classification_df[f'{ar}_{year_style}_period'].unique():
#     classification_df[classification_df[f'{ar}_{year_style}_period']==period]['tci'] = (classification_df[classification_df[f'{ar}_{year_style}_period']==period]['tci'] - classification_df[classification_df[f'{ar}_{year_style}_period']==period]['tci'].min()) / (classification_df[classification_df[f'{ar}_{year_style}_period']==period]['tci'].max() - classification_df[classification_df[f'{ar}_{year_style}_period']==period]['tci'].min())


fiveyears_df_dict = {
    f'{year}': classification_df[classification_df[f'{ar}_{year_style}_period']==f'{year}'][[f'{ar}_{year_style}_period', classification, 'tci']].drop_duplicates(keep='first')\
        for year in classification_df[f'{ar}_{year_style}_period'].unique() if year != f'{year_start}-{year_end}'
}

rank.rank_doubleaxis(fiveyears_df_dict, 
                     rank_num=35, 
                     member_col=classification,
                     value_col='tci',
                     prop_dict={
                                "figsize": (16, 10),
                                "xlabel": "Period",
                                "ylabel": "Technological Fields",
                                "title": "",
                                "fontsize": 15,
                                "year_range": 15,
                                "ascending": False,
                                "color": "default",
    })
